# Workflow Pattern: Parallelization with Async/Await

## 🎯 Objective
Learn how to implement asynchronous parallelization to execute multiple independent AI tasks concurrently, dramatically reducing total execution time.

---

## 🔄 What is Parallelization?

**Parallelization** is an agentic workflow pattern where multiple independent tasks are executed simultaneously rather than sequentially. This pattern is particularly effective for I/O-bound operations like API calls to LLMs.

### Key Benefits:
- ⚡ **Speed**: Multiple API calls execute at the same time
- 💰 **Efficiency**: Better resource utilization
- 📊 **Scalability**: Handles larger workloads effectively

---

## 💡 Common Use Cases

### 1. **RAG with Query Decomposition**
Breaking a complex query into sub-queries, running retrievals for each in parallel, and synthesizing the results.

### 2. **Analyzing Large Documents**
Dividing the document into sections, summarizing each section in parallel, and then combining the summaries.

### 3. **Generating Multiple Perspectives**
Asking multiple LLMs the same question with different persona prompts and aggregating their responses.

### 4. **Map-Reduce Style Operations**
Processing data in parallel chunks and reducing results into a final output.

---

## 🏗️ Architecture Overview

This notebook demonstrates:
- **Async/Await Pattern**: Using Python's `asyncio` for concurrent execution
- **Multiple Expert Agents**: Each agent analyzes a topic from a different perspective
- **Result Aggregation**: Synthesizing individual outputs into a comprehensive report

### Workflow Steps:
1. 📝 Define multiple independent prompts
2. 🚀 Execute all prompts concurrently using `asyncio.gather()`
3. ⏱️ Measure execution time savings
4. 🔄 Aggregate results into a final synthesis

---

**Source**: [Agentic Patterns - Philipp Schmid](https://www.philschmid.de/agentic-pattern)

In [1]:
# ============================================================================
# PARALLEL TASK EXECUTION WITH ASYNC OPENAI
# ============================================================================
# This implementation demonstrates the parallelization pattern using Python's
# asyncio library combined with OpenAI's async client. Multiple LLM calls
# are executed concurrently, significantly reducing total execution time.
# ============================================================================

import asyncio
import time
from openai import AsyncOpenAI
import os
from dotenv import load_dotenv

# Load environment variables (OPENAI_API_KEY)
load_dotenv()

# Initialize the async OpenAI client
# This client supports concurrent API calls using asyncio
openai_client = AsyncOpenAI(api_key=os.getenv("OPENAI_API_KEY"))


async def generate_content(prompt: str, model_name: str = "gpt-4o-mini") -> str:
    """
    Generate content using OpenAI's async API.
    
    This function makes an asynchronous call to OpenAI's chat completions endpoint,
    allowing multiple calls to execute concurrently when used with asyncio.gather().
    
    Args:
        prompt (str): The user prompt to send to the LLM
        model_name (str): The OpenAI model to use (default: "gpt-4o-mini")
    
    Returns:
        str: The generated content from the LLM
    
    Note:
        The 'await' keyword allows other async tasks to execute while waiting
        for the API response, enabling true parallelization.
    """
    # Make async API call to OpenAI
    # Multiple calls to this function can execute simultaneously
    response = await openai_client.chat.completions.create(
        model=model_name,
        messages=[
            {"role": "system", "content": "You are an expert analyst providing concise, well-structured reports."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.7,      # Balanced creativity
        max_tokens=1000       # Sufficient for detailed reports
    )
    
    # Extract and return the generated text
    return response.choices[0].message.content.strip()


async def parallel_tasks(topic: str = "Electric Vehicles (EVs)") -> str:
    """
    Execute multiple expert analysis tasks in parallel and aggregate results.
    
    This function demonstrates the parallelization pattern:
    1. Define multiple independent prompts (different expert perspectives)
    2. Execute all prompts concurrently using asyncio.gather()
    3. Measure time savings compared to sequential execution
    4. Aggregate individual reports into a comprehensive synthesis
    
    Args:
        topic (str): The topic to analyze from multiple perspectives
    
    Returns:
        str: A comprehensive aggregated report synthesizing all expert perspectives
    
    Pattern Benefits:
        - Sequential execution: ~3 API calls × avg_response_time
        - Parallel execution: ~max(individual_response_times)
        - Typical speedup: 2-3x faster for I/O-bound operations
    """
    
    # ========================================================================
    # STEP 1: Define Parallel Tasks
    # ========================================================================
    # Each prompt represents an independent analysis task that can be
    # executed concurrently. The tasks don't depend on each other's results.
    
    prompts = [
        f"As an Economic Analyst, write a concise report on the primary economic impacts of the widespread adoption of {topic}. Focus on job creation, investment trends, and effects on traditional auto and oil industries.",
        
        f"As an Environmental Scientist, write a concise report on the key environmental benefits and challenges associated with {topic}. Consider carbon emissions, battery production/disposal, and electricity grid demands.",
        
        f"As a Policy Advisor, write a concise report summarizing the current government policies, subsidies, and international agreements influencing the market for {topic}."
    ]
    
    print(f"🚀 Starting parallel execution of {len(prompts)} expert analyses...\n")
    
    # ========================================================================
    # STEP 2: Run Tasks Concurrently
    # ========================================================================
    # asyncio.gather() executes all coroutines concurrently and waits for
    # all to complete. This is the key to achieving parallelization.
    
    start_time = time.time()
    
    # Create list of coroutines (tasks that will run concurrently)
    tasks = [generate_content(prompt) for prompt in prompts]
    
    # Execute all tasks in parallel and gather results
    # The 'await' here waits for ALL tasks to complete, but they run simultaneously
    results = await asyncio.gather(*tasks)
    
    end_time = time.time()
    execution_time = end_time - start_time
    
    print(f"⏱️  Time taken: {execution_time:.2f} seconds")
    print(f"📊 Average time per task: {execution_time/len(prompts):.2f} seconds")
    print(f"💡 Sequential execution would have taken: ~{execution_time * 3:.2f} seconds\n")

    # ========================================================================
    # STEP 3: Display Individual Results
    # ========================================================================
    print("=" * 70)
    print("📋 INDIVIDUAL EXPERT REPORTS")
    print("=" * 70 + "\n")
    
    expert_roles = ["Economic Analyst", "Environmental Scientist", "Policy Advisor"]
    for i, result in enumerate(results):
        print(f"┌─ Report from {expert_roles[i]} " + "─" * (70 - len(expert_roles[i]) - 18))
        print(f"│")
        for line in result.split('\n'):
            print(f"│ {line}")
        print(f"└" + "─" * 69)
        print()

    # ========================================================================
    # STEP 4: Aggregate Results into Final Comprehensive Report
    # ========================================================================
    print("=" * 70)
    print("🔄 AGGREGATING RESULTS INTO FINAL REPORT")
    print("=" * 70 + "\n")
    
    # Combine all individual reports for the aggregation prompt
    individual_reports = '\n\n'.join([
        f"Expert Report from {expert_roles[i]}:\n{result}" 
        for i, result in enumerate(results)
    ])

    # Create an aggregation prompt that synthesizes all perspectives
    aggregation_prompt = f"""
You are a senior editor tasked with synthesizing multiple expert reports into a single, cohesive document.

Your task is to create a comprehensive summary report on {topic} that:
1. Has a brief introduction explaining the scope of analysis
2. Synthesizes findings from each domain (economic, environmental, policy)
3. Highlights connections and interactions between different perspectives
4. Concludes with an overall outlook and key takeaways

Here are the expert reports to synthesize:

{individual_reports}

Create a well-structured, comprehensive report that integrates these perspectives.
"""

    # Use OpenAI for the synthesis task
    # This could use a more advanced model if needed
    aggregation_response = await openai_client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a senior editor expert at synthesizing multiple reports into cohesive documents."},
            {"role": "user", "content": aggregation_prompt}
        ],
        temperature=0.7,
        max_tokens=2000  # Higher limit for comprehensive synthesis
    )
    
    return aggregation_response.choices[0].message.content.strip()

In [2]:
# ============================================================================
# EXECUTE PARALLEL WORKFLOW AND DISPLAY FINAL REPORT
# ============================================================================
# Run the parallel analysis workflow and display the synthesized report
# Note: Use 'await' in Jupyter notebooks to run async functions
# ============================================================================

# Define the topic to analyze
topic = "Electric Vehicles (EVs)"

# Execute the parallel workflow
# This will run all expert analyses concurrently and aggregate results
final_report = await parallel_tasks(topic)

# Display the final comprehensive report
print("=" * 70)
print("📊 FINAL COMPREHENSIVE SYNTHESIZED REPORT")
print("=" * 70)
print()
print(final_report)
print()
print("=" * 70)
print("✅ Analysis complete!")
print("=" * 70)

🚀 Starting parallel execution of 3 expert analyses...

⏱️  Time taken: 15.35 seconds
📊 Average time per task: 5.12 seconds
💡 Sequential execution would have taken: ~46.05 seconds

📋 INDIVIDUAL EXPERT REPORTS

┌─ Report from Economic Analyst ────────────────────────────────────
│
│ ### Economic Impacts of Widespread Adoption of Electric Vehicles (EVs)
│ 
│ #### Introduction
│ The transition to Electric Vehicles (EVs) represents a significant shift in the automotive landscape, with profound economic implications. This report outlines the primary impacts of widespread EV adoption focusing on job creation, investment trends, and effects on traditional auto and oil industries.
│ 
│ #### 1. Job Creation
│ - **Manufacturing Jobs**: The EV sector is generating new manufacturing jobs, particularly in battery production and assembly. This shift requires a workforce skilled in advanced technologies, leading to the creation of new training programs and vocational opportunities.
│ - **Infrastructur